# <center> Amazon Fine Foods review: Data Preprocessing

In this notebook, we will clean the data with several steps like,

* Converting score to Positive or negative.


* Removing stop words and duplicate reviews.


* Removing HTML Tags & Punctuation marks.


* Stemming the words using Snowball stemmer.




**************************************************************************************************************************************

Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews

The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.<br>

Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 

Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review


#### Objective:
Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).

<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is nuetral and ignored. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.

In [1]:
#Import section
import numpy as np
import pandas as pd
import sqlite3
import string
import nltk

In [2]:
c=sqlite3.connect('G:\ML\Data_Sets\Amazon\database.sqlite')

In [3]:
filtered_data=pd.read_sql_query(""" select * from reviews where score!=3 """,c)

In [4]:
#Converting scores to Positive or negative

def partition(x):
    if x<3:
        return 'negative'
    else:
        return 'positive'
    
tmp=filtered_data['Score']
new=tmp.map(partition)
new.head()
filtered_data['Score']=new

In [5]:
filtered_data.shape

(525814, 10)

# <center> Data Preprocessing

### Data Deduplication

In [6]:
d=pd.read_sql_query(""" 
select * 
from reviews
where score!=3 and UserId= 'AR5J8UI46CURR'
""",c)
d.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [7]:
sorted_data=filtered_data.sort_values('ProductId',ascending= True,axis=0,na_position='last',inplace=False)

In [8]:
dedup_data=sorted_data.drop_duplicates(subset={"UserId",'ProfileName','Time','Text'},keep='first', inplace= False)
fin_data=dedup_data
dedup_data.shape

(364173, 10)

In [9]:
dedup_data=dedup_data[dedup_data.HelpfulnessNumerator <= dedup_data.HelpfulnessDenominator]
dedup_data.shape

(364171, 10)

# <center> Text preprocessing

In [10]:
#Funcns for clean HTML & Punc

import re

def cleanHtml(s):
    clean=re.compile('<.*?>')
    cleared= re.sub(clean,' ',s)
    return cleared

def cleanPunc(s):
    cleared=re.sub('[?|!|\'|"|(|)|.|#|]',r' ',s)
    return cleared



In [11]:
#Stemming

from nltk.corpus import stopwords

stop = set(stopwords.words('english'))
sno=nltk.stem.SnowballStemmer('english')

print(sno.stem('Tasty'))

tasti


In [12]:
#Program to clean the words

import time
i=0 #Review counter
str=''
filtered_text=[]
pos_words=[]
neg_words=[]
start=time.time()
print('Operation started at : {} '.format(start))
for sentence in dedup_data['Text'].values:
    filtered_sentence=[]
    clean=cleanHtml(sentence)
    clean=cleanPunc(clean)
    for words in clean.split():
        #print(type(words))
        if(len(words)>2 and words.isalpha() and words not in stop):
            stem=(sno.stem(words.lower()))
            filtered_sentence.append(stem)
            if (dedup_data['Score'].values)[i]=='positive':
                pos_words.append(stem)
            if (dedup_data['Score'].values)[i]=='negative':
                neg_words.append(stem)
            
        else:
            continue
            
    str1=' '.join(filtered_sentence)
    filtered_text.append(str1)
    i+=1
    #print('{} reviews completed'.format(i))

dedup_data['New']=filtered_text
end=time.time()-start
print('Time taken = {} '.format(end))


Operation started at : 1537176964.004377 
Time taken = 1348.1416902542114 


C:\Users\Sanat\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
dedup_data.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,New
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,witti littl book make son laugh loud recit car...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew read sendak watch realli rosi movi incorp...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,this fun way children learn month year learn p...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,this great littl book read nice rhythm well go...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,this book poetri month year goe month cute lit...


So in the column 'New' on dataframe dedup_data, we got our data in the cleaned format.

## <center> Saving to a DB file

In [14]:
#Exporting to SQL Lite

conn=sqlite3.connect('G:\ML\Data_Sets\Amazon\Cleaned_AmazonDB.sqlite')
conn.text_factory=str
dedup_data.to_sql('Reviews',conn, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)